<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/lluvia_estaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y estandarización de datasets de lluvia (Quito)